In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader, sampler, Dataset
from sklearn.metrics import roc_curve, auc
import os, time
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import pandas as pd
from src.data import *
from src.utils import save_checkpoint
from torchsummary import summary
from ipywidgets import IntProgress

In [2]:
df = pd.read_csv('./dataframe.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,filename,class,classID,duration,wave
0,angry_8.wav,angry,0,1.856000,[0.52698815 0.52515697 0.52511334 ... 0.525026...
1,angry_9.wav,angry,0,2.026667,[0.51062459 0.51062459 0.5106585 ... 0.511099...
2,angry_45.wav,angry,0,1.920000,[0.51822275 0.51832515 0.51842749 ... 0.521089...
3,angry_44.wav,angry,0,1.173333,[0.47048649 0.47091892 0.47113514 ... 0.453405...
4,angry_40.wav,angry,0,1.216000,[0.44531354 0.44531354 0.44531354 ... 0.445313...


In [3]:
train_loader, valid_loader, test_loader = create_dataloader(df)

----- Custom Dataset -----
train data = {'data': array([[0.56816417],
       [0.56816417],
       [0.56816417],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]]), 'target': 1}


In [9]:
train_loader, valid_loader, test_loader = create_dataloader(df)

----- Custom Dataset -----
train data = {'data': array([[0.54177934],
       [0.54149467],
       [0.54177934],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]]), 'target': 1}


In [10]:
class CNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.kernel_size = 2
        self.stride = 2
        self.input_length = 116584
        # data = [batch_size, in_channels, input_length]
        
        ## layer 1
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=in_dim, out_channels=hidden_dim, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=self.kernel_size, stride=self.stride))
        
        self.out_length = ((self.input_length - self.kernel_size) / self.stride) + 1
        
        ## layer 2
        self.input_length = self.out_length
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_dim, out_channels=out_dim, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        
        self.out_length = ((self.input_length - self.kernel_size) / self.stride) + 1
        
        ## layer 3
        self.layer3 = nn.Sequential(
            nn.ReLU(),
            nn.Linear(int(self.out_length), out_dim),
            nn.Sigmoid())
        
        ## dropout
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = self.layer3(x)
        return x

In [11]:
params = {'in_dim': 1, 
          'hidden_dim': 2 ** 10, 
          'out_dim': 1,
          'lr': 1e-4,
          'num_epoch': 300,
         'print_step': 50}

model = CNN(params['in_dim'], params['hidden_dim'], params['out_dim'])
model

CNN(
  (layer1): Sequential(
    (0): Conv1d(1, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(1024, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): ReLU()
    (1): Linear(in_features=29146, out_features=1, bias=True)
    (2): Sigmoid()
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
val_loss, val_acc = [], []
def valid_model(model, val_loader, roc_auc_best, epoch, train_loader):
    loss_, acc_ = 0., 0.
    y_pred_, y_truth_, pt_ = [], [], []

    for i, data in tqdm(enumerate(val_loader), total=len(val_loader)):
        X = data['data'].reshape(1, 1, -1).to(dtype=torch.float).to(device)
        y = data['target'].float()
            
        logits = model(X).reshape(1).to(device)
        y = y.type_as(logits)
        loss_ += F.binary_cross_entropy_with_logits(logits, y).item()
        pred = logits.ge(0.).byte()
        acc_ += pred.eq(y.byte()).float().mean().item()
        y_pred = torch.sigmoid(logits) 
        # Store batch metrics:
        y_pred_.append(y_pred.tolist())
        y_truth_.append(y.tolist())

    y_pred_ = np.concatenate(y_pred_)
    y_truth_ = np.concatenate(y_truth_)

    val_loss.append(loss_/len(val_loader))
    val_acc.append(acc_/len(val_loader))
    print('Epoch: {}, Validation loss: {:.4f}, Validation accuracy: {:.4f}'.format(epoch, val_loss[epoch-1], val_acc[epoch-1]))
    
    fpr, tpr, _ = roc_curve(y_truth_, y_pred_)
    roc_auc = auc(fpr, tpr)
    s = "VAL ROC AUC: %.4f"%(roc_auc)
    print(s)

    if roc_auc > roc_auc_best:
        roc_auc_best = roc_auc
    
    return roc_auc_best, val_loss, val_acc, fpr, tpr

In [14]:
## training 
train_loss = []
val_loss = []
train_acc = []
val_best_auc = []
total_val_loss = []
total_val_acc = []
total_fpr = []
total_tpr = []
roc_auc_best = 0.5

for epoch in range(params['num_epoch']):
    model.train()
    running_loss, running_acc = 0, 0
    
    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        X = batch['data'].reshape(1, 1, -1).to(dtype=torch.float)
        y = batch['target'].float()
        y_pred = model(X).reshape(1)
        loss = F.binary_cross_entropy_with_logits(y_pred, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        pred = y_pred.ge(0.).byte()
        acc = pred.eq(y.byte()).float().mean()
        running_acc += acc.item()
        
    train_loss.append(running_loss / len(train_loader))
    train_acc.append(running_acc / len(train_loader))
    
    if epoch % params['print_step'] == 0:
        print(f'{epoch} - train loss -- {train_loss[-1]:.3f}, train acc -- {train_acc[-1]:.3f}')
    
    ## validation
    model.eval()
    roc_auc_best, val_loss, val_acc, fpr, tpr = valid_model(model, valid_loader, roc_auc_best, epoch, train_loader)
    val_best_auc.append(roc_auc_best)
    total_val_loss.append(np.mean(val_loss))
    total_val_acc.append(np.mean(val_acc))
    total_fpr.append(fpr)
    total_tpr.append(tpr)
        
    ## save the model
    if epoch % params['print_step'] == 0:
        prefix = f'Conv1d_lr_' + str(params['lr']) + '_' + str(epoch)
        save_checkpoint(model, epoch, train_loss, train_acc, total_val_loss, total_val_acc, fpr, tpr, prefix)
        
    lr_scheduler.step()
    
    output = {'train_loss': train_loss,
             'train_acc': train_acc, 
             'val_loss': val_loss,
             'val_acc': val_acc,
             'total_fpr': total_fpr,
             'total_tpr': total_tpr}

  0%|          | 0/45 [00:00<?, ?it/s]

KeyboardInterrupt: 